# Testing ReAct agents in LMQL and Langchain

In [1]:
import os
os.environ["OPENAI_API_KEY"] = # Add your OpenAI API key here

## Definition of the tools (Wikipedia search and Wikipedia lookup)

In [2]:
from langchain import Wikipedia
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="returns the summary of the Wikipedia page corresponding to the search term. This does not return the full Wikipedia page. Use the Lookup action to get more information about the content of the page"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="returns the next sentence of the current Wikipedia page containing the action input, simulating the Ctrl+F functionality on the browser. If the Wikipedia page does not include the action input, 'No results' will be returned. The action is valid only after a successful Wikipedia search with Search"
    )
]

## ReAct agent with LMQL

In [3]:
from langchain.agents.react.wiki_prompt import EXAMPLES

import lmql
import lmql.runtime.bopenai as bopenai

import nest_asyncio
nest_asyncio.apply()

EXAMPLES = [x.replace("[", "(").replace("]", ")") for x in EXAMPLES]
prompt_start = "\n\n".join(EXAMPLES) 

function = {}
for tool in tools:
    function[tool.name] = tool.func

@lmql.query
async def react(question):
    '''
    argmax(max_len=2048)
        "{prompt_start}\n\nQuestion: {question}"
        for i in range(12):
            "\n[TYPE]: "
            if TYPE == "Thought":
                "[THOUGHT]"
            else:
                "[ACTION]([INPUT])"
                if ACTION == "Finish":
                    break
                "\nObservation: {function[ACTION](INPUT)}"
    from
        "openai/text-davinci-003"
    where
        TYPE in set(["Thought", "Action"]) and
        STOPS_BEFORE(THOUGHT, "\n") and
        ACTION in set(["Finish", "Search", "Lookup"]) and
        STOPS_BEFORE(INPUT, ")")
    '''

In [4]:
question = "Where did the form of music played by Die Rhöner Säuwäntzt originate?"
# The correct answer is: "United States"

In [5]:
num_tokens = bopenai.Completion.get_stats().tokens
num_requests = bopenai.Completion.get_stats().requests
result = (await react(question))
num_tokens2 = bopenai.Completion.get_stats().tokens
num_requests2 = bopenai.Completion.get_stats().requests
num_tokens, delta_tokens = num_tokens2, num_tokens2 - num_tokens
num_requests, delta_requests = num_requests2, num_requests2 - num_requests

print(result[0].prompt[len(prompt_start)+2:])
print(f"\n\n{delta_requests} requests, {delta_tokens} tokens")

Token indices sequence length is longer than the specified maximum sequence length for this model (1482 > 1024). Running this sequence through the model will result in indexing errors


Question: Where did the form of music played by Die Rhöner Säuwäntzt originate?
Thought:  I need to search Die Rhöner Säuwäntzt, find the form of music they play, then find where it originated.
Action: Search(Die Rhöner Säuwäntzt)
Observation: Die Rhöner Säuwäntzt are a skiffle-blues band from Eichenzell-Lütter in Hessen, Germany. The line-up consists of Martin Caba, Christoph Günther  and Christoph Leipold playing  Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties. The expression Säuwäntzt means pork belly and refers also to untidy or unruly children and youth.
Thought:  Die Rhöner Säuwäntzt play Skiffle-Blues. I need to search Skiffle-Blues and find where it originated.
Action: Search(Skiffle-Blues)
Observation: Skiffle is a genre of folk music with influences from American folk music, blues, country, bluegrass, and jazz, generally performed with a mixture of manufactured and homemade or improvised instruments. Originating as a form in the

## ReAct agent with Langchain

In [6]:
from langchain import OpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks import get_openai_callback
from openai import InvalidRequestError

llm = OpenAI(temperature=0, model_name="text-davinci-003")
react_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.REACT_DOCSTORE,
    verbose=True,
    return_intermediate_steps=True
)

with get_openai_callback() as cb:
    result = react_agent({"input": question})
    print(cb)



> Entering new AgentExecutor chain...
Thought: I need to search Die Rhöner Säuwäntzt, find the form of music they play, then find where it originated.
Action: Search[Die Rhöner Säuwäntzt]
Observation: Die Rhöner Säuwäntzt are a skiffle-blues band from Eichenzell-Lütter in Hessen, Germany. The line-up consists of Martin Caba, Christoph Günther  and Christoph Leipold playing  Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties. The expression Säuwäntzt means pork belly and refers also to untidy or unruly children and youth.
Thought: Die Rhöner Säuwäntzt play Skiffle-Blues. I need to search Skiffle-Blues and find where it originated.
Action: Search[Skiffle-Blues]
Observation: Skiffle is a genre of folk music with influences from American folk music, blues, country, bluegrass, and jazz, generally performed with a mixture of manufactured and homemade or improvised instruments. Originating as a form in the United States in the first half of the 20

## ReAct agent with a constraint on tools

In [7]:
@lmql.query
async def react_with_tool_constraint(question):
    '''
    argmax(max_len=2048)
        first = True
        "{prompt_start}\n\nQuestion: {question}"
        for i in range(12):
            "\n[TYPE]: "
            if TYPE == "Thought":
                "[THOUGHT]"
            else:
                if first:
                    "Action: [FIRST_ACTION]([INPUT])"
                    ACTION, first = FIRST_ACTION, false
                else:
                    "Action: [ACTION]([INPUT])"
                if ACTION == "Finish":
                    break
                "\nObservation: {function[ACTION](INPUT)}"
    from
        "openai/text-davinci-003"
    where
        TYPE in set(["Thought", "Action"]) and
        STOPS_BEFORE(THOUGHT, "\n") and
        FIRST_ACTION in set(["Finish", "Search"]) and
        ACTION in set(["Finish", "Search", "Lookup"]) and
        STOPS_BEFORE(INPUT, ")")
    '''

In [8]:
num_tokens = bopenai.Completion.get_stats().tokens
num_requests = bopenai.Completion.get_stats().requests
result = (await react_with_tool_constraint(question))
num_tokens2 = bopenai.Completion.get_stats().tokens
num_requests2 = bopenai.Completion.get_stats().requests
num_tokens, delta_tokens = num_tokens2, num_tokens2 - num_tokens
num_requests, delta_requests = num_requests2, num_requests2 - num_requests

print(result[0].prompt[len(prompt_start)+2:])
print(f"\n\n{delta_requests} requests, {delta_tokens} tokens")

Token indices sequence length is longer than the specified maximum sequence length for this model (1482 > 1024). Running this sequence through the model will result in indexing errors


Failed to call complete endpoint <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0xc3 in position 8457: unexpected end of data
Retrying 19 more times
Question: Where did the form of music played by Die Rhöner Säuwäntzt originate?
Thought:  I need to search Die Rhöner Säuwäntzt, find the form of music they play, then find where it originated.
Action: Action: Search(Die Rhöner Säuwäntzt)
Observation: Die Rhöner Säuwäntzt are a skiffle-blues band from Eichenzell-Lütter in Hessen, Germany. The line-up consists of Martin Caba, Christoph Günther  and Christoph Leipold playing  Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties. The expression Säuwäntzt means pork belly and refers also to untidy or unruly children and youth.
Thought:  Die Rhöner Säuwäntzt play Skiffle-Blues. I need to search Skiffle-Blues and find where it originated.
Action: Action: Search(Skiffle-Blues)
Observation: Skiffle is a genre of folk music with influences fro

## ReAct agent with LMQL with a constraint on a tool's input

In [9]:
prompt_start = """Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny and look up for the eastern sector.
Action: Search(Colorado orogeny,eastern sector)
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and look up for elevation.
Action: Search(High Plains,elevation)
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search(High Plains (United States),elevation)
Observation: (Result 1 / 1) The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).(3)
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish(1,800 to 7,000 ft)

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search(Milhouse,named for)
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish(Richard Nixon)

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups. For this, I will look up for Finnish in the page
Action: Search(Adam Clayton Powell,Finnish)
Observation: Could not find (Adam Clayton Powell). Similar: (’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’).
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search(Adam Clayton Powell (film),Finnish)
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. No results for 'Finnish'.
Thought: Adam Clayton Powell (film) is not a documentary about Finnish rock groups. It must instead be The Saimaa Gesture.
Action: Finish(The Saimaa Gesture)

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common. I will first search Nicholas Ray without lookup words, since the profession is probably included in the search result.
Action: Search(Nicholas Ray,)
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions. I will first search Elia Kazan without lookup words, since the profession is probably included in the search result.
Action: Search(Elia Kazan,)
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish(director, screenwriter, actor)

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search(Arthur’s Magazine,start)
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century. No results for 'start'.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search(First for Women,start)
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.(1) The magazine was started in 1989. No results for 'start'.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish(Arthur’s Magazine)

Question: Were Pavel Urysohn and Leonid Levin known for the same type of work?
Thought: I need to search Pavel Urysohn and Leonid Levin, find their types of work, then find if they are the same. I will search first for Pavel Urysohn, without lookup words since their work will probably be included in the search result.
Action: Search(Pavel Urysohn,)
Observation: Pavel Samuilovich Urysohn (February 3, 1898 - August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory.
Thought: Pavel Urysohn is a mathematician. I need to search Leonid Levin next and check whether he is a mathematician.
Action: Search(Leonid Levin,mathematician)
Observation: (Result 1/1) Leonid Anatolievich Levin (born November 2, 1948) is a Soviet-American mathematician and computer scientist.
Thought: Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.
Action: Finish(yes)

Question: """

In [10]:
def search(s):
    query, lookup = s.split(",")[:2]
    search_result = docstore.search(query)
    if search_result.startswith("Could not find"):
        return search_result
    lookup_result = docstore.lookup(lookup)
    if lookup_result == "No Results":
        lookup_result = f"No results for '{lookup}'."
    return f"{search_result}. {lookup_result}"

function = {"Search": search}

@lmql.query
async def react_with_input_constraint(question):
    '''
    argmax(max_len=2048)
        "{prompt_start}\n\nQuestion: {question}"
        for i in range(12):
            "\n[TYPE]: "
            if TYPE == "Thought":
                "[THOUGHT]"
            else:
                "[ACTION]"
                if ACTION == "Search":
                    "([INPUT1],[INPUT])"
                    "\nObservation: {function['Search'](INPUT1+','+INPUT)}"
                else:
                    "([INPUT])"
                    break
    from
        "openai/text-davinci-003"
    where
        TYPE in set(["Thought", "Action"]) and
        STOPS_BEFORE(THOUGHT, "\n") and
        ACTION in set(["Finish", "Search"]) and
        STOPS_BEFORE(INPUT, ")") and
        STOPS_BEFORE(INPUT1, ",")
        
    '''

In [11]:
result = (await react_with_input_constraint(question))
num_tokens2 = bopenai.Completion.get_stats().tokens
num_requests2 = bopenai.Completion.get_stats().requests
num_tokens, delta_tokens = num_tokens2, num_tokens2 - num_tokens
num_requests, delta_requests = num_requests2, num_requests2 - num_requests

print(result[0].prompt[len(prompt_start)+2:])
print(f"\n\n{delta_requests} requests, {delta_tokens} tokens")

Failed to call complete endpoint <class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0xc3 in position 16248: unexpected end of data
Retrying 19 more times
Question: Where did the form of music played by Die Rhöner Säuwäntzt originate?
Thought:  I need to search Die Rhöner Säuwäntzt and find the origin of the form of music they play.
Action: Search(Die Rhöner Säuwäntzt,origin)
Observation: Die Rhöner Säuwäntzt are a skiffle-blues band from Eichenzell-Lütter in Hessen, Germany. The line-up consists of Martin Caba, Christoph Günther  and Christoph Leipold playing  Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties. The expression Säuwäntzt means pork belly and refers also to untidy or unruly children and youth.. No results for 'origin'.
Thought:  I need to search Skiffle-Blues and find the origin of the form of music.
Action: Search(Skiffle-Blues,origin)
Observation: Skiffle is a genre of folk music with influences from American folk mu